In [1]:
## 라이브러리 불러오기

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

# machine learning
# from sklearn.linear_model import LogisticRegression
# from sklearn.svm import SVC, LinearSVC
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.naive_bayes import GaussianNB
# from sklearn.linear_model import Perceptron
# from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

In [7]:
from sklearn.preprocessing import LabelEncoder

def encode_label(x) : #{
    le    = LabelEncoder()
    le.fit(x)
    label = le.transform(x)
    return label
#}

In [2]:
## 데이터 불러오기
path     = "/Users/yuseon/ys/Kaggle_Practice/data/titanic"
train_df = pd.read_csv(f"{path}/train.csv")
test_df  = pd.read_csv(f"{path}/test.csv")
# combine = [train_df, test_df]

In [3]:
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [5]:
columns = train_df.columns
columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [6]:
label = 'Survived'

## Split data and label
train_x = train_df.drop([label], axis=1)
train_y = train_df[label]
test_x  = test_df.copy()

In [8]:
## Drop columns
drop_columns = ['Name', 'PassengerId', 'Cabin', 'Ticket']
train_x = train_x.drop(drop_columns, axis=1)
test_x  = test_x.drop(drop_columns, axis=1)

In [23]:
## Encode labels that are categorical
enc_label = ['Sex', 'Embarked']
train_x[enc_label] = train_x[enc_label].apply(encode_label)
test_x[enc_label]  = test_x[enc_label].apply(encode_label)

In [34]:
## Divide Age into category
combine = [train_x, test_x]
for dataset in combine:    
    dataset.loc[ dataset['Age'] <= 16, 'Age'] = 0
    dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 32), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'Age'] = 3
    dataset.loc[ dataset['Age'] > 64, 'Age']

# for dataset in combine:
#     dataset["Age"] = dataset.groupby(dataset.Pclass).Age.transform(lambda x: x.fillna(x.median()))

In [35]:
## Fill nan data based on Pclass's median
for dataset in combine : #{
    for c in dataset.columns : #{
        dataset[c] = dataset.groupby(dataset.Pclass)[c].transform(lambda x:x.fillna(x.median()))
    #}
#}

In [39]:
decision_tree = DecisionTreeClassifier()
decision_tree.fit(train_x, train_y)

y_pred = decision_tree.predict(train_x)
print(round(decision_tree.score(train_x, train_y) * 100, 2))

Y_pred = decision_tree.predict(test_x)
## 0.46411

99.21


In [43]:
submission = pd.DataFrame({
        "PassengerId": test_df["PassengerId"],
        "Survived": Y_pred
    })
submission.to_csv('./submission.csv', index=False)